In [ ]:
from random import random, choice, randint
from functools import reduce
from math import ceil
from collections import namedtuple
from dataclasses import dataclass
import numpy as np
from copy import copy
from pprint import pprint

In [ ]:
PROBLEM_SIZE = 50
NUM_SETS = 100
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [ ]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [ ]:
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost


def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost


fitness = fitness2

### Problem Parameters

Having ``POPULATION_SIZE``=30 and ``OFFSPRING_SIZE``=20 we are making a Steady state Genetic Algorithm which means that we are creating 20 new individuals to put in the population, we mix and keep the 30 best ones and then throw them away 20.

In [ ]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15

Il genoma dovrebbe essere la stessa cosa di uno stato in un path search, esso quindi indica gli stati presi.

- ``select_parent`` is a function that takes ``TOURNAMENT_SIZE`` elements from the population randomly and returns the best of them, the ``champion``, depending on their ``fitness``.
- ``mutation`` is a kind of tweak function, it creates a new individual. In this case, as in single states, we simply change a random value of the genotype. This means that they change a state to taken from not taken or vice versa.
- ``one_cut_xover`` takes two individuals and produces one. The one-cut technique was used, i.e., a random value is chosen which will be the index of the cut of the parent genotypes. The new genotype will therefore be formed by the sum of the parent genotypes where the first part will be from the first individual and the second part will be from the second individual who have both entered.

In [ ]:
@dataclass
class Individual:
    genotype: list[bool]
    fitness: tuple

def select_parent(population): 
    pool = [choice(population) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutation(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None, genotype=ind1.genotype[:cut_point]+ind2.genotype[cut_point:])
    return offspring

# Inizialize the population

In [ ]:
population = [Individual(genotype=[choice((True, False)) for _ in range(NUM_SETS)], fitness=None) for _ in range(POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genotype)

- Now that we have the population the first step is to take the parents in each ``generation``

- In Genetic Algorithms we have two choices: the first is to select mutation or recombination, in the second we do recombination and with a certain probability we mutate. We choose the first, which is also the one recommended in general, so only one of the two is done in each cycle. In case of mutation I pick one parent, in case of crossover I pick two.

In [ ]:
for generation in range(10):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:
            #mutation
            p=select_parent(population)
            o = mutation(p)
        else:
            # crossover
            p1=select_parent(population)
            p2=select_parent(population)
            o=one_cut_xover(p1, p2)
        offspring.append(o)
        
    for i in offspring:
        i.fitness = fitness(i)
        population.extend(offspring)
    # survival selection
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    pprint(population[0].fitness)